# Лабораторная работа №4 Генерация текстов на основе LSTM

## Импорт библиотек

In [6]:
import re

import numpy as np
import pandas as pd


import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator

from sklearn.model_selection import train_test_split

import sqlite3
import zipfile

from tqdm.auto import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kirillanpilov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Device

In [7]:
device = torch.device("cpu")

## Функция обучения

In [27]:
def train(model: nn.Module, crterion, optimizer , n_epochs, train_loader, test_loader):
    loss_train = []
    accuracy_train = []

    for epoch in range(n_epochs):
        model.train()
        for batch in tqdm(train_loader , desc=f'Training epoch {epoch + 1}:'):
            inputs = batch['input']
            labels = batch['label']
            output = model(inputs)
            loss = criterion(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        model.eval()
        total = 0
        correct = 0
        for batch in tqdm(test_loader , desc=f'Testing epoch {epoch + 1}:'):
              inputs = batch['input']
              labels = batch['label']
              output = model(inputs)
              _, predicted = torch.max(output.data, 1)
              labels = torch.nonzero(labels == 1)[:,1]

              total += labels.size(0)
              correct += (predicted == labels).sum().item()

        test_accuracy = correct / total
        accuracy_train.append(test_accuracy)

        print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
        loss_train.append(loss.item())
        generate_text(model, "я люблю ", num = 100)


## Задание 1. Загрузите текст из произведений Ницше
('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt').


Выведете следующее:
- А) длину всего корпуса;
- Б) количество предложений;
- В) сколько всего символов используется?


In [8]:
with open('nietzsche.txt', 'r') as file:
    lines = file.read()

text_nietzsche = lines
text_nietzsche[:100]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all ph'

In [9]:
text_nietzsche = text_nietzsche.lower()
text_nietzsche = re.sub(r'\s', ' ', text_nietzsche) # replace \n \t and etc.
text_nietzsche = re.sub(r'\s{2,}', ' ', text_nietzsche) # replace repeated whitespace

### Длина всего корпуса

In [10]:
len(text_nietzsche)

598861

### Количество предложений

In [11]:
len(sent_tokenize(text_nietzsche))

2864

### Cколько всего символов используется?

In [12]:
len(set(text_nietzsche))

56

## Задание 2. Сократите текст наполовину избыточными последовательностями символов maxlen

In [13]:
vocab = build_vocab_from_iterator(text_nietzsche, specials=["<unk>"])
token2inx = vocab.get_stoi()
inx2token = {inx: token for token, inx in vocab.get_stoi().items()}

In [14]:
token2inx

{'é': 55,
 'æ': 54,
 '[': 49,
 '7': 48,
 'ä': 53,
 ']': 50,
 '8': 46,
 '4': 44,
 '2': 41,
 'z': 40,
 "'": 39,
 '9': 51,
 ')': 36,
 '(': 35,
 '?': 34,
 'q': 32,
 'j': 31,
 ';': 30,
 '1': 37,
 ':': 29,
 '6': 47,
 '3': 43,
 'r': 9,
 's': 8,
 'd': 12,
 'ë': 56,
 '=': 38,
 'k': 26,
 'n': 7,
 'h': 10,
 'u': 14,
 '0': 52,
 'p': 17,
 ',': 21,
 '"': 27,
 't': 3,
 'i': 4,
 '-': 24,
 '5': 45,
 'a': 5,
 'e': 2,
 'v': 23,
 'o': 6,
 '<unk>': 0,
 '.': 25,
 'c': 13,
 'f': 15,
 'm': 16,
 '!': 33,
 'l': 11,
 'g': 18,
 '_': 42,
 'y': 19,
 'w': 20,
 'b': 22,
 ' ': 1,
 'x': 28}

In [15]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text_nietzsche) - maxlen, step):
    sentences.append(text_nietzsche[i: i + maxlen])
    next_chars.append(text_nietzsche[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(token2inx)))
y = np.zeros((len(sentences), len(token2inx)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, token2inx[char]] = 1
    y[i, token2inx[next_chars[i]]] = 1

nb sequences: 199607
Vectorization...


In [16]:
x.shape

(199607, 40, 57)

In [17]:
sentences[0]

'preface supposing that truth is a woman-'

In [18]:
x[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

У нас в датасете 199607 40 символьных отрезков и ответ следущая буква. В каждом sample 40 списков one-hot, где одна единица обозначает букву.

## Задание 3. Создайте модель LSTM для генерации текста.

- А) Напишите вспомогательную функцию для выборки индекса из массива вероятностей
- Б) Напишите функцию, которая будет вызываться в конце каждой эпохи и печатать сгенерированный текст
- В) Запустите модель на обучение Имейте ввиду, что требуется не менее 20 эпох, прежде чем сгенерированный текст начнет звучать связно. Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные сети требуют довольно больших вычислительных затрат.
- Г) Проверьте работу модели в онлайн режиме.

### Train and test

In [19]:
X_train, X_test , y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [20]:
len(X_train) , len(X_test) , len(y_train) , len(y_test)

(159685, 39922, 159685, 39922)

### Dataset

In [18]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input = torch.Tensor(self.data[idx]).to(device)
        label = torch.Tensor(self.labels[idx]).to(device)
        sample = {'input': input, 'label': label}
        return sample

In [22]:
train_dataset= CustomDataset(X_train, y_train)

test_dataset= CustomDataset(X_test, y_test)

### DataLoader

In [23]:
train_dataloader = DataLoader(train_dataset,  batch_size=256)

test_dataloader = DataLoader(test_dataset, batch_size=256)

### Функция для генерации текста

In [19]:
def generate_text(model, text, num = 100):
      model.eval()
      for i in range(num):
        x = torch.zeros(len(text), vocab_size).to(device)
        for j, char in enumerate(text):
          x[j, token2inx[char]] = 1
        x = x.unsqueeze(0)


        output = model(x)
        _, predicted = torch.max(output.data, 1)

        text += inx2token[int(predicted)]
      print(text)


### Модель

In [20]:
class AnpilovGpt(nn.Module):
    def __init__(self, vocab_size, hidden_size1, hidden_size2 ):
        super().__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_size1,  batch_first=True) # (batch_size, seq_len, input_size)
        self.linear = nn.Linear(in_features = hidden_size1 , out_features = hidden_size2 )
        self.projection = nn.Linear(in_features = hidden_size2 , out_features = vocab_size) # vocab_size = num_classes

        self.dropout = nn.Dropout(p = 0.25) # defult p = 0.5
        self.tanh = nn.Tanh() # against vanish gradient

    def forward(self, input):
        output, _ = self.lstm(input)  # [batch_size, seq_len, hidden_dim]
        output = output[:, -1]
        output = self.dropout(self.linear(self.tanh(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.tanh(output))
        return projection


### Проверка модели на свой текст

### Обучение

In [26]:
n_epochs = 20
vocab_size = len(vocab) # 57
hidden_size1 = 256
hidden_size2 = 128


model = AnpilovGpt(vocab_size, hidden_size1, hidden_size2).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters()) # lr = 0.001


train(model, criterion, optimizer, n_epochs, train_dataloader, test_dataloader)

Training epoch 1::   0%|          | 0/624 [00:00<?, ?it/s]

Testing epoch 1::   0%|          | 0/156 [00:00<?, ?it/s]

Epoch [1/20], Loss: 2.3763, Test Accuracy: 32.18%
i love the the the the the the the the the the the the the the the the the the the the the the the the the 


Training epoch 2::   0%|          | 0/624 [00:00<?, ?it/s]

Testing epoch 2::   0%|          | 0/156 [00:00<?, ?it/s]

...

Testing epoch 19::   0%|          | 0/156 [00:00<?, ?it/s]

Epoch [19/20], Loss: 0.8525, Test Accuracy: 52.40%
i love of fact that every stronger as a called exere such a tendous of something of mankind is a sort of su


Training epoch 20::   0%|          | 0/624 [00:00<?, ?it/s]

Testing epoch 20::   0%|          | 0/156 [00:00<?, ?it/s]

Epoch [20/20], Loss: 0.8334, Test Accuracy: 52.09%
i love of for his philosophers have look work soul what in the power of spirits and sometom and others in a


### Обучение с dropout = 0.25

In [27]:
n_epochs = 20
vocab_size = len(vocab) # 57
hidden_size1 = 256
hidden_size2 = 128


model = AnpilovGpt(vocab_size, hidden_size1, hidden_size2).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters()) # lr = 0.001


train(model, criterion, optimizer, n_epochs, train_dataloader, test_dataloader)

Training epoch 1::   0%|          | 0/624 [00:00<?, ?it/s]

Testing epoch 1::   0%|          | 0/156 [00:00<?, ?it/s]

Epoch [1/20], Loss: 2.4513, Test Accuracy: 31.90%
i love the the the the the the the the the the the the the the the the the the the the the the the the the 


Training epoch 2::   0%|          | 0/624 [00:00<?, ?it/s]

Testing epoch 2::   0%|          | 0/156 [00:00<?, ?it/s]

...

Testing epoch 19::   0%|          | 0/156 [00:00<?, ?it/s]

Epoch [19/20], Loss: 1.0076, Test Accuracy: 53.12%
i love of the conduct the deerers of the artists of his properiting the postions of the souls of the genera


Training epoch 20::   0%|          | 0/624 [00:00<?, ?it/s]

Testing epoch 20::   0%|          | 0/156 [00:00<?, ?it/s]

Epoch [20/20], Loss: 0.9581, Test Accuracy: 52.50%
i love of the conduct therefore, that is to say that the sense of the consciences is so far allo of the age


### Проверка модели в онлайн режиме.

In [28]:
text = "what a day"
generate_text(model, text, 1000)

what a day been position of the conduct therefore, that is to say that the sense of the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the agesome the consciences is so far allo of the ages

## Задание 4. Создайте самостоятельно генерацию текста для РУССКОЯЗЫЧНОГО НАБОРА глав Wikibooks.
Полный текст Wikibooks содержит более 270000 глав на 12 языках https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/data


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [8]:
zf = zipfile.ZipFile('archive.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
conn = sqlite3.connect('wikibooks.sqlite')

cursor = conn.cursor()

cursor.execute("SELECT * from ru")

raw_eng_text = cursor.fetchall()

In [30]:
columns = [i[0] for i in cursor.description]
data = pd.DataFrame(raw_eng_text, columns=columns)

cursor.close()
conn.close()
text = " ".join(data['body_text'])

In [31]:
text = text.lower()
text = re.sub(r'[^а-яА-ЯёЁ ]', '', text)
text = re.sub(r'\s', ' ', text) # replace \n \t and etc.
text = re.sub(r'\s{2,}', ' ', text) # replace repeated whitespace
text[:100]

'рабочая станциясерверперсональный компьютер в википедии имеется статья по теме свидетельство частног'

In [32]:
len(text)

38837958

In [33]:
vocab = build_vocab_from_iterator(text, specials=["<unk>"])
token2inx = vocab.get_stoi()
inx2token = {inx: token for token, inx in vocab.get_stoi().items()}

In [34]:
token2inx

{'щ': 31,
 'ш': 30,
 'ф': 29,
 'э': 32,
 'ю': 28,
 'ё': 33,
 'ж': 26,
 'х': 25,
 'й': 24,
 'г': 22,
 'я': 17,
 'д': 15,
 'б': 21,
 'п': 14,
 'м': 13,
 'ъ': 34,
 ' ': 1,
 'к': 12,
 'з': 19,
 'л': 11,
 'ч': 23,
 'и': 4,
 'в': 10,
 'т': 7,
 'ь': 20,
 'у': 16,
 'н': 6,
 'а': 5,
 'р': 8,
 'о': 2,
 'ц': 27,
 'е': 3,
 'ы': 18,
 'с': 9,
 '<unk>': 0}

In [35]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    if len(sentences) > 1_000_000:
      break
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(token2inx)))
y = np.zeros((len(sentences), len(token2inx)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, token2inx[char]] = 1
    y[i, token2inx[next_chars[i]]] = 1

nb sequences: 1000001
Vectorization...


In [36]:
X_train, X_test , y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [37]:
train_dataset = CustomDataset(X_train, y_train)

test_dataset = CustomDataset(X_test, y_test)

In [38]:
train_dataloader = DataLoader(train_dataset,  batch_size=256)

test_dataloader = DataLoader(test_dataset, batch_size=256)

In [39]:
n_epochs = 20
vocab_size = len(vocab) # 57
hidden_size1 = 256
hidden_size2 = 128


model_wiki = AnpilovGpt(vocab_size, hidden_size1, hidden_size2).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_wiki.parameters()) # lr = 0.001


train(model_wiki, criterion, optimizer, n_epochs, train_dataloader, test_dataloader)

Training epoch 1::   0%|          | 0/3125 [00:00<?, ?it/s]

Testing epoch 1::   0%|          | 0/782 [00:00<?, ?it/s]

Epoch [1/20], Loss: 1.8251, Test Accuracy: 45.19%
я люблю по совтретельности по совтретельности по совтретельности по совтретельности по совтретельности по со


Training epoch 2::   0%|          | 0/3125 [00:00<?, ?it/s]

Testing epoch 2::   0%|          | 0/782 [00:00<?, ?it/s]

...

Testing epoch 19::   0%|          | 0/782 [00:00<?, ?it/s]

Epoch [19/20], Loss: 1.2240, Test Accuracy: 56.38%
я люблю и то в консульсами на стандартной проблемам на сотрудникам должно быть создавать в составе одной и с


Training epoch 20::   0%|          | 0/3125 [00:00<?, ?it/s]

Testing epoch 20::   0%|          | 0/782 [00:00<?, ?it/s]

Epoch [20/20], Loss: 1.1847, Test Accuracy: 56.31%
я люблю и то в составе от которых выделяюте в структуре предприятия в течение периода которые можно получить


In [43]:
text = "физика это "
generate_text(model_wiki, text, 1000)

физика это может быть на разностью момент но в разделе проведения измерений в процессе с помощью применения конструкций для проведения измерений в течение части продукт по проверка на выражению с помощью проведения измерений в течение части продаж по возможности поставщику на разрешении с помощью применения конструкций для проведения измерений в течение части продукт по проверка на выражению с помощью проведения измерений в течение части продаж по возможности поставщику на разрешении с помощью применения конструкций для проведения измерений в течение части продукт по проверка на выражению с помощью проведения измерений в течение части продаж по возможности поставщику на разрешении с помощью применения конструкций для проведения измерений в течение части продукт по проверка на выражению с помощью проведения измерений в течение части продаж по возможности поставщику на разрешении с помощью применения конструкций для проведения измерений в течение части продукт по проверка на выражению с 